# Treinando um modelo Word2Vec com livros do Harry Potter

**Word2Vec** é um método para gerar *Word Embeddings* a partir de um corpus de texto, utilizando redes neurais.

Desenvolvido por Tomas Mikolov *et al.* (Google) em 2013, é um dos métodos de geração de *word embeddings* mais populares em tarefas de processamento de linguagem natural (PLN) como análise de sentimento, tradução de textos e reconhecimento de entidades nomeadas (NER).


Para exercitar, vamos treinar um modelo **Word2Vec** com os livros do *Harry Potter*, escritos por J. K. Rowling. 

Os livros foram retirados [deste repositório](https://github.com/priyanks179/harry-word2vec/tree/master/harry_txt) e traduzidos para português.

### Passo 1 - importanto as bibliotecas
Vamos primeiro instalar e importar as bibliotecas que utilizaremos.

In [3]:
import re
import numpy as np
from gensim import corpora, models, similarities
import nltk
import pickle
import pandas as pd
import unicodedata
import spacy 

Para o pré-processamento em língua portuguesa, vamos usar o pacote ```pt_core_news_sm```. Instale antes com:
    
```python -m spacy download pt_core_news_sm```

In [ ]:
nlp = spacy.load('pt_core_news_sm') 

### Passo 2 - Pré-processamento do *corpus*
Aqui eu traduzi os 7 livros do repositório citado acima e concatenei todos em um único arquivo TXT. Retirei os pontos usados em abreviaturas (ex Sr. e Sra.) e fiz um split por ". ", de forma que cada frase fique em uma única linha.

Vamos abrir o arquivo e transformá-lo em um *dataframe*.

In [4]:
file = open(r"data\harry-potter-pt.txt", "r", encoding='UTF-8')
df = pd.DataFrame(file)
df.columns = ['lines']
df = df.sort_index()
file.close()

Assim são os textos.

In [24]:
df

,lines
0,Harry Potter E A pedra do feiticeiro byJ K Ro...
1,"r e a Sra Dursley, do número quatro, Rua dos A..."
2,Eles foram as últimas pessoas que você esperar...
3,O Sr Dursley era o diretor de uma empresa cha...
4,Ele era um homem grande e musculoso com quase ...
...,...
68162,"Enquanto Harry olhava para ela, ele abaixou a..."
68163,Eu sei que ele vai. \n
68164,A cicatriz não doía a Harry há dezenove anos. \n
68165,Tudo foi bem. \n


Vamos primeiro remover os acentos;

In [32]:
def remove_accents(text):
    '''Strip accents out.'''
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

clean2 = lambda x: cleaning1(x)

In [33]:
df = pd.DataFrame(df.lines.apply(remove_accents))

In [34]:
df

,lines
0,Harry Potter E A pedra do feiticeiro byJ K Ro...
1,"r e a Sra Dursley, do numero quatro, Rua dos A..."
2,Eles foram as ultimas pessoas que voce esperar...
3,O Sr Dursley era o diretor de uma empresa cha...
4,Ele era um homem grande e musculoso com quase ...
...,...
68162,"Enquanto Harry olhava para ela, ele abaixou a..."
68163,Eu sei que ele vai. \n
68164,A cicatriz nao doia a Harry ha dezenove anos. \n
68165,Tudo foi bem. \n


Agora, vamos limpar, usando expressões regulares, tudo que não é caracter alfanumérico e passar tudo para caixa baixa.


In [35]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['lines'])

In [36]:
brief_cleaning

<generator object <genexpr> at 0x000002479C1FAE60>

Vamos lematizar e retirar as *stopwords*, para diminuir a dimensionalidade, com a biblioteca ```spacy```. Vamos usar ```spacy.pipe()```.

In [5]:
def cleaning(doc):
    # lematizar e retirar stopwords
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # como Word2Vec usa palavras de contexto para aprender a representação vetorial de uma palavra-alvo,
    # se uma frase tiver apenas uma ou duas palavras,
    # o benefício para o treinamento é muito pequeno    
    if len(txt) > 2:
        return ' '.join(txt)

In [37]:
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

In [38]:
txt[68162:68167]

['  harry olhar abaixar o mao distraidamente e tocar o cicatriz raiar testo',
 None,
 '  o cicatriz nao doia o harry ha dezenove ano',
 None,
 '  o paginar']

In [39]:
txt2 = [str(row.strip()) for row in txt if (row != None and row.strip() != '.')]

In [42]:
txt2[63263:63268]

['o mao harry levantar despedir',
 'ficar murmurar ginny',
 'harry olhar abaixar o mao distraidamente e tocar o cicatriz raiar testo',
 'o cicatriz nao doia o harry ha dezenove ano',
 'o paginar']

Vamos removee os valores faltantes e duplicados.

In [43]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean

,clean
0,harry potter e o pedrar feiticeiro byj k row...
1,r e o sr dursley numerar ruir alfeneiro orgulh...
2,ultimar pessoa voce esperar estar envolvido es...
3,o sr dursley o diretor empresar chamar grunn...
4,homem e musculoso quase nenhum pescoco ter bigode
...,...
68159,o trem dobrar esquinar
68160,o mao harry levantar despedir
68161,ficar murmurar ginny
68162,harry olhar abaixar o mao distraidamente e t...


In [41]:
len(txt2)

63268

#### Bigramas
O pacote ```Gensim Phrases``` detecta automaticamente frases comuns (bigramas) de uma lista de frases.

Veja mais em: https://radimrehurek.com/gensim/models/phrases.html

Com o método ```Phrases()```, vamos criar frases relevantes a partir da lista de frases.


In [44]:
from gensim.models.phrases import Phrases, Phraser

sent = [row.split() for row in df_clean['clean']]

In [45]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

```Phraser()``` reduz o consumo de memória de ```phrases``` ao descartar estado do modelo não necessário para a detecção de bigramas.

In [46]:
bigram = Phraser(phrases)

Transformamos o corpus com base nos bigramas detectados.

In [47]:
sentences = bigram[sent]

#### Palavras frequentes
Vamos calcular a frequencia das palavras, para verificar a eficácia da lematização, remoção de palavras irrelevantes e adição de bigramas.

In [49]:
from collections import defaultdict 

word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

14595

In [50]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['o', 'e', 'harry', 'nao', 'dizer', 'voce', 'ron', 'hermione', 'ter', 'olhar']

### Passo 3 - Treinamento do modelo

Hora de treinar nosso modelo Word2Vec com nossos dados. Primeiro, vamos verificar o ambiente.

In [51]:
import multiprocessing
from gensim.models import Word2Vec
from time import time

cores = multiprocessing.cpu_count() # conta o número de núcleos do computador

Vamos parametrizar nosso modelo.

- *min_count*: Ignora todas as palavras com frequência absoluta total inferior a esta
- *window*: A distância máxima entre a palavra atual e a prevista em uma frase. Por exemplo, palavras da janela à esquerda e palavras da janela à esquerda do nosso alvo
- *size*: Dimensionalidade dos vetores
- *sample*: O limite para configurar quais palavras de alta frequência são reduzidas aleatoriamente
- *alpha*: A taxa de aprendizagem inicial
- *min_alpha*: A taxa de aprendizado cairá linearmente para *min_alpha* conforme o treinamento progride
- *negative*: Se > 0, a amostragem negativa será usada, o int para negativo especifica quantas "palavras de ruído" devem ser eliminadas. Se definido como 0, nenhuma amostra negativa é usada
- *workers*: Quantidade de *threads* de trabalho para treinar o modelo (treinamento mais rápido com máquinas multicore)

In [52]:
w2v_model = Word2Vec(min_count=3,
                     window=2,
                     size=32,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=10,
                     workers=cores-1)

#### Construindo o vocabulário e treinando o modelo
Parâmetros do treinamento:

*total_examples: Contagem de sentenças;
*epochs*: Número de iterações (épocas) no corpus

In [53]:
w2v_model.build_vocab(sentences, progress_per=10000)


In [54]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


(8076257, 19146480)

### Passo 4 - Salvando o modelo
Vamos salvar o modelo nos formatos *KeyedVectors* e binário, para utilizarmos posteriormente.

In [88]:
w2v_model.init_sims(replace=True)  # deixa o modelo mais eficiente, pois não vamos mais treiná-lo futuramente

In [55]:
w2v_model.save("word2vec-harry-potter.model")
w2v_model.wv.save_word2vec_format('model-harry-potter.bin', binary=True)



Agora que já treinamos nosso modelo, [vamos ver aqui como utilizá-lo](https://github.com/lisaterumi/word2vec-harry-potter-portugues/blob/main/%5B2%5D%20tSNE-Harry-Potter.ipynb).